<a href="https://colab.research.google.com/github/TempestConnor/Coding-101/blob/main/HSR_SPD_Sim_Clara_Topaz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Very scuffed, super user-unfriendly Python script for simulating full team AV with support for dynamic action advances and speed buffs. Make sure to check comments on how to use and what to change.

Make a copy > Runtime > Run All to sim.

If there is a problem contact Sitri in HSR KQM.

In [ ]:
import copy

Don't touch anything from here until the next notification

In [ ]:
class Character:
    def __init__(self, name,base_spd, static_spd, AG = 10000):
        self.name = name
        self.spd = static_spd
        self.static = static_spd
        self.base = base_spd
        self.action_count = 0
        self.AG = 10000
        self.status_queue = []
        self.AV = 0

    def spd_change(self, mult, flat):
        self.spd += self.base * mult + flat

    def action_change(self, change):
        print(f"{self.name} action is changed by {change*100}%. From {max(0,self.AG)} to ", end="")
        self.AG = max(0, self.AG - 10000*change)
        print(max(0,self.AG))
        print(" ")

    def spd_reset(self):
        self.spd = self.static

    def receive(self, status):
        temp = copy.deepcopy(status)
        self.status_queue.append(temp)
        self.spd += self.base*status.mult + status.flat
        print(f"{status.caster} casted SPD effect {status.mult*100}% SPD and {status.flat} SPD Flat on {self.name} for {status.duration} turns. Current SPD: {int(self.spd)}")
        print("")

    def expire(self, status):
        self.spd = self.spd - (self.base*status.mult + status.flat)
        print(f"SPD effect {status.mult*100}% SPD and {status.flat} SPD Flat on {self.name} from {status.caster} expired. Current SPD: {int(self.spd)}")
        print("")
        self.status_queue.remove(status)


    def AG_reset(self):
        self.AG = 10000

    def act(self):
        if self.AG <= 0:
            for status in self.status_queue[:]:
                status.duration -=1
                if status.duration == 0:
                    self.expire(status)
            self.AG_reset()
        self.AG = max(0,self.AG - self.spd)
        self.AV = int(self.AG/self.spd)

In [ ]:
class SPD_Effect:
    def __init__(self, duration, value, activation_turn, target = list,caster = str):
        self.duration = duration
        self.mult = value[0]
        self.flat = value[1]
        self.caster = caster
        self.target = target
        self.activate = activation_turn

In [ ]:
class Action_Effect:
    def __init__(self, value, activation_turn, target = list, caster = str):
        self.value = value
        self.target = target
        self.caster = caster
        self.activate = activation_turn

In [ ]:
class Cycle:
    def __init__(self, AV = 100):
        self.AV = AV

    def prog(self):
        self.AV-=1

Stuff down here can be changed, but only in parts with comments

In [ ]:
# don't touch
def battle_time(count):
    cycle_arr = []
    for i in range(count):
        if i == 0:
            cycle_arr.append(Cycle(150))
        else:
            cycle_arr.append(Cycle())
    return cycle_arr

"""
For team, format is "name" : Character("name", base_spd, starting_spd)
"""
def create_team():
    team_dict = {
        "Topaz": Character("Topaz", 110, 155),
        "Lynx": Character("Lynx", 100, 148),
        "Clara": Character("Clara", 90, 96),
        "Tingyun": Character("Tingyun", 106, 158),


        "Numby": Character("Numby", 80, 80)
    }
    return team_dict

def create_enemy():
    enemy_dict = {
        "Enemy 1": Character("Enemy 1", 120, 201)
    }
    return enemy_dict

def battle(count):
    cycle_arr = battle_time(count)
    team = create_team()
    enemy = create_enemy()
    on_field = team | enemy
    """
    For speed buffs/debuffs, add into queue in format:

    SPD_Effect(duration, (mult value, flat value),
    turn to activate based on caster, [character(s) receiving buffs], caster)

    For action/gauge changes, add into queue in format:

    Action_Effect(value, turn to activate based on caster,
    [character(s) receiving buffs], caster)

    Make sure recipient is always in []. To add entire team, can replace
    [character 1, character 2,...] with list(team.keys())

    """
    buff_queue  = [#SPD_Effect(1, (0.12,0), 2, list(team.keys()), "Tingyun"), #hackerspace
                   #SPD_Effect(1, (0.12,0), 1, list(team.keys()), "Tingyun"), #hackerspace
                   #SPD_Effect(1, (0.12,0), 5, list(team.keys()), "Tingyun"), #hackerspace
                   #SPD_Effect(1, (0.12,0), 4, list(team.keys()), "Tingyun"), #hackerspace
                   #SPD_Effect(1, (0.12,0), 8, list(team.keys()), "Tingyun"), #hackerspace
                   #SPD_Effect(1, (0.12,0), 7, list(team.keys()), "Tingyun"), #hackerspace
                   #SPD_Effect(1, (0.2,0), 2, ["Lynx"], "Tingyun"), #Tingyun E1
                   #SPD_Effect(1, (0.2,0), 5, ["Lynx"], "Tingyun"), #Tingyun E1
                   #SPD_Effect(1, (0.2,0), 8, ["Lynx"], "Tingyun"), #Tingyun E1
                   #SPD_Effect(2, (0,51), 1, list(team.keys()), "Tingyun"), #Tingyun Ult
                   #SPD_Effect(2, (0,51), 4, list(team.keys()), "Tingyun"), #Tingyun Ult
                   #SPD_Effect(2, (0,51), 7, list(team.keys()), "Tingyun"), #Tingyun Ult
                   #SPD_Effect(1, (0.2,0), 1, ["Tingyun"], "Tingyun"), #Tingyun Trace
                   #SPD_Effect(1, (0.2,0), 4, ["Tingyun"], "Tingyun"), #Tingyun Trace
                   #SPD_Effect(1, (0.2,0), 7, ["Tingyun"], "Tingyun"), #Tingyun Trace
                   #Action_Effect(0.40,0, ["Tingyun"], "Tingyun"), #Vonwacq
                   #SPD_Effect(1, (0.20,0),1, ["Tingyun"], "Tingyun"), #Patience
                   #SPD_Effect(1, (0.20,0),4, ["Tingyun"], "Tingyun"), #Patience
                   #SPD_Effect(1, (0.20,0),7, ["Tingyun"], "Tingyun"), #Patience

                   #SPD_Effect(1, (0.20,0),1, ["Clara"], "Tingyun"), #Patience
                   #SPD_Effect(1, (0.20,0),4, ["Clara"], "Tingyun"), #Patience
                   #SPD_Effect(1, (0.20,0),7, ["Clara"], "Tingyun"), #Patience

                   #Action_Effect(0.40,0, ["Bronya"], "Bronya"), #Vonwacq
                   #SPD_Effect(99, (0.25,0),7, ["Seele"], "Seele"), #Seele spd buff


                   #SPD_Effect(2,(0,50),1, list(team.keys()),"Tingyun"), #Tingyun 3t ult
                   #SPD_Effect(2,(0,50),3, list(team.keys()),"Tingyun"),
                    #SPD_Effect(2,(0,50),5, list(team.keys()),"Tingyun"),
                    #SPD_Effect(2,(0,50),7, list(team.keys()),"Tingyun"),


                   #Cycle 0
                   Action_Effect(0.50,1, ["Numby"], "Topaz"),
                   Action_Effect(0.50,1, ["Numby"], "Enemy 1"), #March counter


                   Action_Effect(0.50,1, ["Numby"], "Lynx"), #Ult
                   Action_Effect(0.50,1, ["Numby"], "Clara"), #Ult
                   #Impossible action -- Action_Effect(0.50,1, ["Numby"], "Tingyun"), #Ult

                   Action_Effect(0.50,5, ["Numby"], "Lynx"), #Ult
                   Action_Effect(0.50,5, ["Numby"], "Tingyun"), #Ult

                   Action_Effect(0.50,9, ["Numby"], "Lynx"), #Ult
                   Action_Effect(0.50,9, ["Numby"], "Tingyun"), #Ult

                   Action_Effect(0.50,2, ["Numby"], "Topaz"),
                   Action_Effect(0.50,2, ["Numby"], "Enemy 1"), #March counter


                   Action_Effect(0.50,3, ["Numby"], "Topaz"),
                   Action_Effect(0.50,3, ["Numby"], "Enemy 1"), #March counter

                   Action_Effect(0.50,5, ["Numby"], "Topaz"),
                   Action_Effect(0.50,5, ["Numby"], "Enemy 1"), #March counter


                   Action_Effect(0.50,6, ["Numby"], "Topaz"),
                   Action_Effect(0.50,6, ["Numby"], "Enemy 1"), #March counter

                   Action_Effect(0.50,7, ["Numby"], "Topaz"),
                   Action_Effect(0.50,7, ["Numby"], "Enemy 1"), #March counter

                   Action_Effect(0.50,8, ["Numby"], "Topaz"),
                   Action_Effect(0.50,8, ["Numby"], "Enemy 1"), #March counter

                   Action_Effect(0.50,9, ["Numby"], "Topaz"),
                   Action_Effect(0.50,9, ["Numby"], "Enemy 1"), #March counter

                   Action_Effect(0.50,10, ["Numby"], "Topaz"),
                   Action_Effect(0.50,10, ["Numby"], "Enemy 1"), #March counter

                   Action_Effect(0.50,11, ["Numby"], "Topaz"),
                   Action_Effect(0.50,11, ["Numby"], "Enemy 1"), #March counter


                   Action_Effect(0.50,12, ["Numby"], "Topaz"),
                   Action_Effect(0.50,12, ["Numby"], "Enemy 1"), #March counter

                   Action_Effect(0.50,13, ["Numby"], "Topaz"),
                   Action_Effect(0.50,13, ["Numby"], "Enemy 1"), #March counter

                   Action_Effect(0.50,4, ["Numby"], "Topaz"),
                   Action_Effect(0.50,4, ["Numby"], "Enemy 1"), #March counter

                   Action_Effect(0.50,14, ["Numby"], "Topaz"),
                   Action_Effect(0.50,14, ["Numby"], "Enemy 1"), #March counter





                   Action_Effect(-0.25, 2,["Enemy 1"], "Enemy 1")
                   ]

    # Don't touch this
    for i in range(len(cycle_arr)):
        print(f"Cycle {i+1}")
        while cycle_arr[i].AV >= 0:
            on_field = dict(sorted(on_field.items(), key=lambda item : item[1].spd, reverse= True))
            for char in on_field.values():
                if char.AG <= 0:
                    char.action_count+=1
                    print(f"{cycle_arr[i].AV} - {char.name} - Action count: {char.action_count} - {int(char.spd)} SPD")
                    # for temp_char in team.values():
                    #     print(f"{temp_char.name} : {temp_char.AV}")
                    print(" ")

                char.act()

                for buff in buff_queue[:]:
                    if char.name == buff.caster and char.action_count == buff.activate:
                        if type(buff) == SPD_Effect:
                            for target in buff.target:
                                on_field[target].receive(buff)
                        if type(buff) == Action_Effect:
                            for target in buff.target:
                                on_field[target].action_change(buff.value)
                        buff_queue.remove(buff)
            cycle_arr[i].prog()
        print("")

battle(6) # change the number to how many cycles you want to simulate

Cycle 1
100 - Enemy 1 - Action count: 1 - 201 SPD
 
Numby action is changed by 50.0%. From 6000 to 1000.0
 
87 - Numby - Action count: 1 - 80 SPD
 
86 - Tingyun - Action count: 1 - 158 SPD
 
85 - Topaz - Action count: 1 - 155 SPD
 
Numby action is changed by 50.0%. From 9840 to 4840.0
 
82 - Lynx - Action count: 1 - 148 SPD
 
Numby action is changed by 50.0%. From 4600.0 to 0
 
82 - Numby - Action count: 2 - 80 SPD
 
50 - Enemy 1 - Action count: 2 - 201 SPD
 
Numby action is changed by 50.0%. From 7440 to 2440.0
 
Enemy 1 action is changed by -25.0%. From 9799 to 12299.0
 
45 - Clara - Action count: 1 - 96 SPD
 
Numby action is changed by 50.0%. From 2040.0 to 0
 
45 - Numby - Action count: 3 - 80 SPD
 
22 - Tingyun - Action count: 2 - 158 SPD
 
20 - Topaz - Action count: 2 - 155 SPD
 
Numby action is changed by 50.0%. From 8000 to 3000.0
 
14 - Lynx - Action count: 2 - 148 SPD
 

Cycle 2
88 - Enemy 1 - Action count: 3 - 201 SPD
 
Numby action is changed by 50.0%. From 360.0 to 0
 
88 